<a href="https://colab.research.google.com/github/TranThiDieuHien/pythonbasics/blob/master/huong_dan_gia_dinh_ANOVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy import stats
import pandas as pd
from scipy.stats import bartlett

PHÂN TÍCH ANOVA MỘT CHIỀU

Phân tích ANOVA chấp nhận các giả định sau:

(1) Tất cả các nhóm đều độc lập với nhau

(2) Dữ liệu của mỗi nhóm đều tuân theo phân phối chuẩn. Tính chất này có thể được kiểm định bằng phương pháp Shapiro-Wilks.

(3) Phương sai (variance) của các nhóm bằng nhau. Tính chất này có thể được kiểm định bằng phương pháp kiểm định Bartlett hay Levene.

Nếu dữ liệu hiện có thỏa mãn các tính chất trên thì chúng ta mới tiến hành kiểm định ANOVA. Do vậy trước khi tiến hành kiểm định ANOVA, chúng ta cần thực hiện kiểm tra tập dữ liệu cần thỏa mãn 3 tính chất trên.
    

In [2]:
#Bước 1: Hãy tạo một tập dữ liệu mẫu sample_1 với kích thước 100 tuân theo phân phối chuẩn, cho mean = 20, std = 0.2
sample_1 = stats.norm.rvs(loc=20, scale=0.2, size=100)
print(sample_1[:10])

[19.84962092 20.15072642 19.94631238 20.32074767 20.0083364  19.94969638
 20.00898569 20.15982641 19.81301079 19.89151181]


In [3]:
#Bước 2: hãy tạo tập dữ liệu mẫu sample_2 với kích thước 100 tuân theo phân phối Poisson
sample_2 = stats.poisson.rvs(loc=20, mu = 1, size = 100)
print(sample_2[:10])

[20 23 22 20 20 21 21 20 21 20]


Hãy tham khảo các hướng dẫn sau để kiểm định tính chất thứ 2 bằng phương pháp Shapiro-Wilks đối với sample_1 và sample_2

(1) [Tài liệu tham khảo scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html)

(2) [Hướng dẫn cụ thể của statology](https://www.statology.org/shapiro-wilk-test-python/)

In [4]:
#Code kiểm định tính chất thứ 2 của sample_1 và sample_2
# thử nghiệm Shapiro-Wilk để xác định xem một mẫu có đến từ phân phối chuẩn hay không.
print("Kiểm định Shapiro-Wilk để xem mẫu có tuân theo phân phối chuẩn")
shapiro_test1 = stats.shapiro(sample_1) 
print("Statistic =", shapiro_test1[0], "\n",
      "p value =", shapiro_test1[1])
print("-----------------------------------------")

print("Kiểm định Shapiro-Wilk để xem mẫu có tuân theo phân phối Poisson")
shapiro_test2 = stats.shapiro(sample_2)
print("Statistic =", shapiro_test2[0], "\n",
      "p value =", shapiro_test2[1])

Kiểm định Shapiro-Wilk để xem mẫu có tuân theo phân phối chuẩn
Statistic = 0.9935709238052368 
 p value = 0.9199907183647156
-----------------------------------------
Kiểm định Shapiro-Wilk để xem mẫu có tuân theo phân phối Poisson
Statistic = 0.7888218760490417 
 p value = 1.1394363924610573e-10


Hãy tham khảo các hướng dẫn sau để kiểm định tính chất thứ 3 bằng kiểm định Bartlett hoặc kiểm định Levene

(1) [Hướng dẫn tổng quát](https://www.marsja.se/levenes-bartletts-test-of-equality-homogeneity-of-variance-in-python/)

(2) [Hướng dẫn ở stackoverflow](https://stackoverflow.com/questions/36141254/how-to-test-for-homoscedasticity-having-the-same-population-variance-in-python)

(3) Sử dụng bài giảng về kiểm định Bartlett và Levene ở buổi trước

In [5]:
#Code kiểm định tính đồng nhất về phương sai của các nhóm

Example: An University have 4 Majors: Business, Psych, Math, English. We will consider the level of frustration of 100 students with each major.

- $\mu_{1}$ : level of frustration of Business Majors
- $\mu_{2}$ : level of frustration of English Majors
- $\mu_{3}$ : level of frustration of Math Majors
- $\mu_{4}$ : level of frustration of Psych Majors

The size of each sample is 35. Assume that Y(level of frustration) = 10

In [6]:
#Sinh ngẫu nhiên
Business = np.random.choice(10, 35)
Math = np.random.choice(10, 35)
English = np.random.choice(10, 35)
Psych = np.random.choice(10, 35)
#Tạo dataframe
list_of_tuples = list(zip(Business, English, Math, Psych))
df2 = pd.DataFrame(list_of_tuples, columns = ["Business Majors", "English Majors", "Math Majors", "Psych Majors"])
print(df2.head())
#Lấy từng nhóm
Bus = df2['Business Majors']
Eng = df2['English Majors']
Math = df2['Math Majors']
Psy = df2['Psych Majors']
alpha = 0.05


   Business Majors  English Majors  Math Majors  Psych Majors
0                9               0            7             2
1                3               2            6             5
2                7               1            2             4
3                6               1            4             4
4                9               5            7             0


In [7]:
#Kiểm định bằng bartlett
stat, pvalue = stats.bartlett(Bus, Eng, Math, Psy)
print("Statistic =", stat, "\n",
      "p value =", pvalue)
if pvalue < alpha:
  print("Chưa có bằng chứng thống kê để bác bỏ H0")
else:
  print("Có bằng chứng thống kê để bác bỏ H0")
  #Xem phương sai của từng biến
  b, e, m, p = [np.var(x, ddof=1) for x in [Bus, Eng, Math, Psy]]
  print("PS của Bus =", b, "\n",
      "PS của Eng =", e, "\n",
      "PS của Math =", m, "\n",
      "PS của Psy =", p, "\n",)

Statistic = 0.21516997868737997 
 p value = 0.975103971869142
Có bằng chứng thống kê để bác bỏ H0
PS của Bus = 9.349579831932777 
 PS của Eng = 8.067226890756302 
 PS của Math = 8.652100840336134 
 PS của Psy = 9.137815126050421 



In [8]:
#Kiểm định bằng Levene
stat1, pvalue1 = stats.levene(Bus, Eng, Math, Psy)
print("Statistic =", stat1, "\n",
      "p value =", pvalue1)
if pvalue1 < alpha:
  print("Chưa có bằng chứng thống kê để bác bỏ H0")
else:
  print("Có bằng chứng thống kê để bác bỏ H0")
  #Xem phương sai của từng biến
  b, e, m, p = [np.var(x, ddof=1) for x in [Bus, Eng, Math, Psy]]
  print("PS của Bus =", b, "\n",
      "PS của Eng =", e, "\n",
      "PS của Math =", m, "\n",
      "PS của Psy =", p, "\n",)

Statistic = 0.35442424242424253 
 p value = 0.7860085039920268
Có bằng chứng thống kê để bác bỏ H0
PS của Bus = 9.349579831932777 
 PS của Eng = 8.067226890756302 
 PS của Math = 8.652100840336134 
 PS của Psy = 9.137815126050421 

